In [8]:
import pandas as pd

In [9]:
INPUT_DIR = "./Datasets"
calendar = pd.read_csv(f"{INPUT_DIR}/calendar.csv")
selling_prices = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv")
sample_submission = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv")
sales_train_val = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv")


In [10]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [11]:
# Get columns starting with 'd_'
date_cols = [col for col in sales_train_val.columns if col.startswith("d_")]

# Melt (unpivot) the dataframe
sales_train_val_melted = pd.melt(
    sales_train_val,
    id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
    value_vars=date_cols,
    var_name="date",
    value_name="quantity",
)
print(sales_train_val_melted.shape)
print(sales_train_val_melted.columns)
sales_train_val_melted.head()

(58327370, 8)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date',
       'quantity'],
      dtype='object')


,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [12]:
# Merge sales_train_val_melted with only the necessary columns from calendar
sales_train_val_melted_with_wm_yr_wk = sales_train_val_melted.merge(
    calendar[["d", "wm_yr_wk"]], how="left", left_on="date", right_on="d"
)

# Drop the redundant 'd' column if you don't need it
sales_train_val_melted_with_wm_yr_wk = sales_train_val_melted_with_wm_yr_wk.drop(
    "d", axis=1
)
sales_train_val_melted_with_wm_yr_wk.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101


In [13]:
selling_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [14]:
sales_train_val_melted_with_wm_yr_wk.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101


In [15]:
# Merge the DataFrames on store_id, item_id, and wm_yr_wk
sales_train_val_melted_with_prices = pd.merge(
    sales_train_val_melted_with_wm_yr_wk,
    selling_prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left",
)

# Display the merged DataFrame
sales_train_val_melted_with_prices.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,NaN


In [16]:
# Check missing prices
missing_prices = sales_train_val_melted_with_prices["sell_price"].isna()

# Analyze corresponding sales values
print("Summary of sales values where prices are missing:")
print(sales_train_val_melted_with_prices.loc[missing_prices, "quantity"].value_counts())
print("\nTotal missing prices:", missing_prices.sum())
print("Of which:")
print(
    f"- Zero sales: {(sales_train_val_melted_with_prices.loc[missing_prices, 'quantity'] == 0).sum()}"
)
print(
    f"- Non-zero sales: {(sales_train_val_melted_with_prices.loc[missing_prices, 'quantity'] > 0).sum()}"
)

# Show a few examples of rows with missing prices but non-zero sales
print("\nExample rows with missing prices but non-zero sales:")
print(
    sales_train_val_melted_with_prices[
        missing_prices & (sales_train_val_melted_with_prices["quantity"] > 0)
    ].head()
)

Summary of sales values where prices are missing:
quantity
0    12299413
Name: count, dtype: int64

Total missing prices: 12299413
Of which:
- Zero sales: 12299413
- Non-zero sales: 0

Example rows with missing prices but non-zero sales:
Empty DataFrame
Columns: [id, item_id, dept_id, cat_id, store_id, state_id, date, quantity, wm_yr_wk, sell_price]
Index: []


In [17]:
# Assuming merged_data is the DataFrame containing the sell_price column
sales_train_val_melted_with_prices["sell_price"] = sales_train_val_melted_with_prices[
    "sell_price"
].fillna(0)

# Display the updated DataFrame
sales_train_val_melted_with_prices.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0


In [18]:
sales_train_val_melted_with_prices["revenue"] = (
    sales_train_val_melted_with_prices["sell_price"]
    * sales_train_val_melted_with_prices["quantity"]
)
sales_train_val_melted_with_prices.head()


,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk,sell_price,revenue
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0,0.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,0.0,0.0


In [19]:
# Group by store_id and date, calculating total revenue
total_revenue = (
    sales_train_val_melted_with_prices.groupby(["store_id", "date"])["revenue"]
    .sum()
    .reset_index()
)

# Rename the column for clarity
total_revenue.rename(columns={"sell_price": "total_revenue"}, inplace=True)

# Display the grouped DataFrame
total_revenue

,store_id,date,revenue
0,CA_1,d_1,10933.16
1,CA_1,d_10,7704.54
2,CA_1,d_100,8997.65
3,CA_1,d_1000,9300.40
4,CA_1,d_1001,11259.55
...,...,...,...
19125,WI_3,d_995,8751.16
19126,WI_3,d_996,9635.44
19127,WI_3,d_997,6156.77
19128,WI_3,d_998,5933.57


In [20]:
total_revenue_merged_calendar = total_revenue.merge(
    calendar[["d", "date"]], how="left", left_on="date", right_on="d"
).drop(columns=["d"])
total_revenue_merged_calendar


,store_id,date_x,revenue,date_y
0,CA_1,d_1,10933.16,2011-01-29
1,CA_1,d_10,7704.54,2011-02-07
2,CA_1,d_100,8997.65,2011-05-08
3,CA_1,d_1000,9300.40,2013-10-24
4,CA_1,d_1001,11259.55,2013-10-25
...,...,...,...,...
19125,WI_3,d_995,8751.16,2013-10-19
19126,WI_3,d_996,9635.44,2013-10-20
19127,WI_3,d_997,6156.77,2013-10-21
19128,WI_3,d_998,5933.57,2013-10-22


In [23]:
total_revenue_merged_calendar_sorted = total_revenue_merged_calendar.sort_values(
    by=["store_id", "date_y"], ascending=True
)
total_revenue_merged_calendar_sorted


,store_id,date_x,revenue,date_y
0,CA_1,d_1,10933.16,2011-01-29
1025,CA_1,d_2,9787.06,2011-01-30
1136,CA_1,d_3,7201.38,2011-01-31
1247,CA_1,d_4,7407.74,2011-02-01
1358,CA_1,d_5,6566.12,2011-02-02
...,...,...,...,...
18228,WI_3,d_1909,9341.89,2016-04-20
18230,WI_3,d_1910,9212.23,2016-04-21
18231,WI_3,d_1911,10959.56,2016-04-22
18232,WI_3,d_1912,13120.79,2016-04-23


In [29]:
pivot = total_revenue_merged_calendar_sorted.pivot(
    index="date_x", columns="store_id", values="revenue"
)
pivot


store_id,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
date_x,,,,,,,,,,
d_1,10933.16,9101.52,11679.83,4561.59,6586.68,9915.78,7597.99,6454.72,5451.46,9367.88
d_10,7704.54,5987.10,11940.49,3985.17,5456.29,7874.00,5920.59,3788.93,4688.76,7818.83
d_100,8997.65,7717.91,11745.96,3950.04,5912.75,7758.90,6054.28,3562.83,3370.61,6784.60
d_1000,9300.40,7867.20,15008.62,6126.41,6582.39,9177.12,7116.20,7075.66,8217.59,5956.10
d_1001,11259.55,9413.64,15962.89,5957.78,6857.85,10979.24,7904.82,8686.22,9929.11,7229.63
...,...,...,...,...,...,...,...,...,...,...
d_995,15414.63,12630.59,20618.58,7104.56,9690.33,13944.37,9912.63,10145.85,10676.81,8751.16
d_996,17549.38,12936.40,22215.00,8017.31,10508.35,13679.02,10126.05,9130.72,9962.94,9635.44
d_997,11332.70,7732.41,17793.25,7010.68,8125.99,10326.53,8148.24,6888.60,7938.68,6156.77


In [30]:
pivot.min()

store_id
CA_1    0.00
CA_2    3.16
CA_3    0.00
CA_4    0.00
TX_1    0.00
TX_2    0.00
TX_3    1.58
WI_1    0.00
WI_2    0.00
WI_3    0.00
dtype: float64

In [31]:
pivot.idxmin()

store_id
CA_1    d_1062
CA_2     d_697
CA_3    d_1427
CA_4    d_1062
TX_1    d_1427
TX_2    d_1792
TX_3    d_1062
WI_1    d_1427
WI_2    d_1062
WI_3     d_331
dtype: object

In [32]:
filtered = sales_train_val_melted_with_prices[
    sales_train_val_melted_with_prices["store_id"] == "TX_3"
]
filtered

,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk,sell_price,revenue
18294,HOBBIES_1_001_TX_3_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_3,TX,d_1,0,11101,0.00,0.00
18295,HOBBIES_1_002_TX_3_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,TX_3,TX,d_1,0,11101,0.00,0.00
18296,HOBBIES_1_003_TX_3_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,TX_3,TX,d_1,0,11101,0.00,0.00
18297,HOBBIES_1_004_TX_3_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_3,TX,d_1,0,11101,0.00,0.00
18298,HOBBIES_1_005_TX_3_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,TX_3,TX,d_1,0,11101,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
58318218,FOODS_3_823_TX_3_validation,FOODS_3_823,FOODS_3,FOODS,TX_3,TX,d_1913,2,11613,2.98,5.96
58318219,FOODS_3_824_TX_3_validation,FOODS_3_824,FOODS_3,FOODS,TX_3,TX,d_1913,0,11613,2.48,0.00
58318220,FOODS_3_825_TX_3_validation,FOODS_3_825,FOODS_3,FOODS,TX_3,TX,d_1913,0,11613,3.98,0.00
58318221,FOODS_3_826_TX_3_validation,FOODS_3_826,FOODS_3,FOODS,TX_3,TX,d_1913,2,11613,1.28,2.56


In [34]:
filtered = filtered[filtered["date"] == "d_1062"]
filtered

,id,item_id,dept_id,cat_id,store_id,state_id,date,quantity,wm_yr_wk,sell_price,revenue
32368184,HOBBIES_1_001_TX_3_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_3,TX,d_1062,0,11348,8.38,0.0
32368185,HOBBIES_1_002_TX_3_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,TX_3,TX,d_1062,0,11348,3.97,0.0
32368186,HOBBIES_1_003_TX_3_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,TX_3,TX,d_1062,0,11348,0.00,0.0
32368187,HOBBIES_1_004_TX_3_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_3,TX,d_1062,0,11348,4.64,0.0
32368188,HOBBIES_1_005_TX_3_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,TX_3,TX,d_1062,0,11348,2.98,0.0
...,...,...,...,...,...,...,...,...,...,...,...
32371228,FOODS_3_823_TX_3_validation,FOODS_3_823,FOODS_3,FOODS,TX_3,TX,d_1062,0,11348,2.68,0.0
32371229,FOODS_3_824_TX_3_validation,FOODS_3_824,FOODS_3,FOODS,TX_3,TX,d_1062,0,11348,2.68,0.0
32371230,FOODS_3_825_TX_3_validation,FOODS_3_825,FOODS_3,FOODS,TX_3,TX,d_1062,0,11348,3.98,0.0
32371231,FOODS_3_826_TX_3_validation,FOODS_3_826,FOODS_3,FOODS,TX_3,TX,d_1062,0,11348,1.28,0.0


In [36]:
filtered.idxmax()

id            32369795
item_id       32369795
dept_id       32369281
cat_id        32368749
store_id      32368184
state_id      32368184
date          32368184
quantity      32370634
wm_yr_wk      32368184
sell_price    32368242
revenue       32370634
dtype: int64

In [38]:
filtered.loc[32370634, :]


id            FOODS_3_226_TX_3_validation
item_id                       FOODS_3_226
dept_id                           FOODS_3
cat_id                              FOODS
store_id                             TX_3
state_id                               TX
date                               d_1062
quantity                                1
wm_yr_wk                            11348
sell_price                           1.58
revenue                              1.58
Name: 32370634, dtype: object

In [28]:
# total_revenue_merged_calendar_sorted.to_excel("total_revenue_by_store_date.xlsx")
